In [25]:
from oaklib.implementations.pronto.pronto_implementation import ProntoImplementation
from oaklib.resource import OntologyResource
import pandas as pd
#from pandasql import sqldf
import duckdb
from scipy import stats

In [26]:
yeast_annot = pd.read_csv("../4932.tab",sep="\t",header=0)

In [27]:
yeast_annot.head()

,locusId,accession,GI,scaffoldId,start,stop,strand,sysName,name,desc,COG,COGFun,COGDesc,TIGRFam,TIGRRoles,GO,EC,ECDesc
0,6580634,NP_009332.1,6319249.0,223723,2169,1807,-,YAL068C,PAU8,Pau8p (RefSeq),NaN,NaN,NaN,NaN,NaN,GO:0006950,NaN,NaN
1,6580635,NP_878038.1,33438754.0,223723,2480,2707,+,YAL067W-A,YAL067W-A,Putative protein of unknown function; identifi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6580636,NP_009333.1,6319250.0,223723,9017,7236,-,YAL067C,SEO1,"Putative permease, member of the allantoate tr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6580637,NP_009335.1,6319252.0,223723,11952,11566,-,YAL065C,YAL065C,Putative protein of unknown function; has homo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6580638,NP_009336.1,6319253.0,223723,12047,12427,+,YAL064W-B,YAL064W-B,Fungal-specific protein of unknown function (R...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#yeast_annot.query("GO like %GO:0006950%")
genes_stress_response_data = duckdb.query("SELECT * FROM yeast_annot WHERE GO LIKE '%GO:0006950%'").df()

In [29]:
gene_ids = genes_stress_response_data['sysName']
gene_ids

0       YAL068C
1       YAR020C
2     YBL108C-A
3       YBR067C
4       YBR301W
5       YCR104W
6       YDR542W
7       YEL049W
8       YER011W
9       YFL020C
10      YFL014W
11      YGL261C
12      YGR294W
13      YHL046C
14      YIL176C
15      YIL011W
16      YIR041W
17      YJL223C
18      YJR150C
19      YJR151C
20      YKL224C
21      YLL064C
22      YLL025W
23      YLR037C
24      YLR040C
25      YLR461W
26      YMR325W
27      YNR076W
28      YOL161C
29      YOR009W
30      YOR010C
31      YOR394W
32      YPL282C
Name: sysName, dtype: object

In [30]:
oi = ProntoImplementation(OntologyResource(local=False, slug='go.obo'))

In [38]:
rels = oi.get_outgoing_relationships_by_curie('GO:0006950')
for rel, parents in rels.items():
    print(f'  {rel} ! {oi.get_label_by_curie(rel)}')
    for parent in parents:
        print(f'    {parent} ! {oi.get_label_by_curie(parent)}')

  rdfs:subClassOf ! subClassOf
    GO:0050896 ! response to stimulus


In [39]:
labels = gene_ids
hypotheses = ["GO:0006950"]
hypotheses

['GO:0006950']

In [33]:
#removed: inferred_types=None,
def enrichment_test(subjects=None, background=None, hypotheses=None,  ontology_labels=None, threshold=0.05, labels=False, direction='greater'):
        """
        Performs term enrichment analysis.

        Arguments
        ---------

        subjects: string list

            Sample set. Typically a gene ID list. These are assumed to have associations

        background: string list

            Background set. If not set, uses full set of known subject IDs in the association set

        threshold: float

            p values above this are filtered out

        labels: boolean

            if true, labels for enriched classes are included in result objects

        direction: 'greater', 'less' or 'two-sided'

            default is greater - i.e. enrichment test. Use 'less' for depletion test.

        """
        if subjects is None:
            subjects = []

        subjects=set(subjects)
        bg_count = {}
        sample_count = {}
        potential_hypotheses = set()
        sample_size = len(subjects)
        #for s in subjects:
        #    potential_hypotheses.update(inferred_types(s))
        #if hypotheses is None:
        #    hypotheses = potential_hypotheses
        #else:
        #    hypotheses = potential_hypotheses.intersection(hypotheses)
        print("Hypotheses: {}".format(hypotheses))

        # get background counts
        # TODO: consider doing this ahead of time
        if background is None:
            background = set(subjects)
        else:
            background = set(background)

        # ensure background includes all subjects
        background.update(subjects)

        bg_size = len(background)

        for c in hypotheses:
            bg_count[c] = 0
            sample_count[c] = 0
            
        #hard-coded
        bg_count["GO:0006950"] = len(gene_ids)
        sample_count["GO:0006950"] = len(gene_ids)
        
        #ADD BACK IN LATER
        #for s in background:
        #    ancs = inferred_types(s)
        #    for a in ancs.intersection(hypotheses):
        #        bg_count[a] = bg_count[a]+1
        #for s in subjects:
        #    for a in inferred_types(s):
        #        if a in hypotheses:
        #            sample_count[a] = sample_count[a]+1

        hypotheses = [x for x in hypotheses if bg_count[x] > 1]
        print("Filtered hypotheses: {}".format(hypotheses))
        num_hypotheses = len(hypotheses)
        results = []
        for cls in hypotheses:

            # https://en.wikipedia.org/wiki/Fisher's_exact_test
            #
            #              Cls  NotCls    RowTotal
            #              ---  ------    ---
            # study/sample [a,      b]    sample_size
            # rest of ref  [c,      d]    bg_size - sample_size
            #              ---     ---
            #              nCls  nNotCls

            a = sample_count[cls]
            b = sample_size - a
            c = bg_count[cls] - a
            d = (bg_size - bg_count[cls]) - b
            #logger.debug("ABCD="+str((cls,a,b,c,d,sample_size)))
            _, p_uncorrected = stats.fisher_exact( [[a, b], [c, d]], direction)
            p = p_uncorrected * num_hypotheses
            if p>1.0:
                p=1.0
            #logger.debug("P={} uncorrected={}".format(p,p_uncorrected))
            if p<threshold:
                res = {'c':cls,'p':p,'p_uncorrected':p_uncorrected}
                if labels:
                    res['n'] = ontology.label(cls)
                results.append(res)

        results = sorted(results, key=lambda x:x['p'])
        return results

In [40]:
## Run enrichment tests using all classes in ontology
enr = enrichment_test(subjects=gene_ids, background = None, hypotheses=hypotheses, threshold=0.1, labels=True)

Hypotheses: ['GO:0006950']
Filtered hypotheses: ['GO:0006950']


In [41]:
enr

[]

In [36]:
## Show first 20 results
for r in enr[:20]:
        print("{:8.3g} {} {:40s}".format(r['p'],r['c'],str(r['n'])))